In [6]:
import random
import numpy as np
import statistics

In [2]:
class Mutationen:
    def __init__(self):
        pass

    def two_swap_mutation(self, individual, K):
        size = len(individual)
        if size < 2:
            return individual # Keine Mutation möglich, wenn die Länge weniger als 2 ist

        for _ in range(K):  # Wähle zwei verschiedene Positionen zufällig aus
            pos1, pos2 = random.sample(range(size), 2)
            individual[pos1], individual[pos2] = individual[pos2], individual[pos1] # Tausch der Werte an den beiden Positionen

        return individual

    def one_translocation_mutation(self, individual, K):
        size = len(individual)
        for _ in range(K):  # Wähle eine zufällige Position für das Gen, das verschoben werden soll
            pos1 = random.randint(0, size - 1)
            gene = individual.pop(pos1)
            pos2 = random.randint(0, size - 1) # Wähle eine neue zufällige Position, um das Gen wieder einzufügen
            individual.insert(pos2, gene)
        return individual

    def mutate(self, individual, method, K):
        if method == '2-Swap':
            return self.two_swap_mutation(individual, K)
        elif method == '1-Translocation':
            return self.one_translocation_mutation(individual, K)
        else:
            raise ValueError("Invalid mutation method. Choose '2-Swap' or '1-Translocation'.")

In [3]:
class Rekombination:
    def __init__(self):
        pass

    def order_based_crossover(self, parent1, parent2):
        size = len(parent1)
        start, end = sorted(random.sample(range(size), 2))  # Wähle zwei zufällige Schnittpunkte
        #Initailisieren der Kinder
        child1 = [-1] * size
        child1[start:end] = parent1[start:end]
        #Aktuelle Indexposition 
        child1_pointer = end
        parent2_pointer = end
        #OBX für 1.Kind
        while -1 in child1:
            if parent2[parent2_pointer % size] not in child1: #Index Bsp [5 % 9]: Modulo Operator, hier 5<9 --> parent2[5] mit Kind verglichen
                child1[child1_pointer % size] = parent2[parent2_pointer % size] #Indexpositon des Kindes wird mit entsprechnden Eintrag des Eltern aufgefüllt
                child1_pointer += 1
            parent2_pointer += 1

        return child1

    def partially_mapped_crossover(self, parent1, parent2):
        size = len(parent1)
        start, end = sorted(random.sample(range(size), 2))
        child1 = [-1] * size
        child1[start:end] = parent1[start:end]

        mapping = {}
        for i in range(start, end):
            mapping[parent1[i]] = parent2[i]
            mapping[parent2[i]] = parent1[i]

        def fill_child(child, parent, start, end, mapping):
            for i in range(size):
                if i < start or i >= end:
                    gene = parent[i]
                    while gene in child[start:end]:
                        gene = mapping[gene]
                    child[i] = gene

        fill_child(child1, parent2, start, end, mapping)
        return child1

    def keine_rekombination(self, parent1, parent2): #Keine Rekombination --> Kinder sind gleich zu Eltern
        return parent1

    def recombine(self, parent1, parent2, method):
        if method == 'OBX':
            return self.order_based_crossover(parent1, parent2)
        elif method == 'PMX':
            return self.partially_mapped_crossover(parent1, parent2)
        elif method == 'NIX':
            return self.keine_rekombination(parent1, parent2)
        else:
            raise ValueError("Invalid recombination method. Choose 'OBX', 'PMX', or 'NIX'.")

In [4]:
class Zielfunktionen:
    def __init__(self):
        pass

    def ziel_1(self, kandidat): #Maximum - Minimum 
        sums = self.evaluate(kandidat)
        return max(sums) - min(sums)  

    def ziel_2(self, kandidat): #Summe der absoluten Differenz von allen Paaren
        sums = self.evaluate(kandidat)
        total_difference = 0
        for i in range(len(sums)):  #Sums enthält alle möglichen s1 bis s8 Elemente
            for j in range(i + 1, len(sums)):  #Durchläuft nachfolgendes Element nach i 
                total_difference += abs(sums[i] - sums[j])
        return total_difference

    def ziel_3(self, kandidat):  #Summe der absoluten Differenz von s1 und allen andern s2 bis s8
        sums = self.evaluate(kandidat)
        s1 = sums[0]
        total_difference = 0
        for i in range(1, len(sums)):
            total_difference += abs(s1 - sums[i])
        return total_difference

    def evaluate(self, kandidat):
        s1 = sum(kandidat[0:3])
        s2 = sum(kandidat[3:6])
        s3 = sum(kandidat[6:9])
        s4 = sum(kandidat[0:9:3])
        s5 = sum(kandidat[1:9:3])
        s6 = sum(kandidat[2:9:3])
        s7 = kandidat[0] + kandidat[4] + kandidat[8]
        s8 = kandidat[2] + kandidat[4] + kandidat[6]
        return [s1, s2, s3, s4, s5, s6, s7, s8]

In [17]:

class EvolutionaryAlgorithm:
    def __init__(self, mu, lamb, mutation_k, mutation_method, recombination_method, zielfunktion,max_evaluations=10000):  #Weil Probleme, versuch mit maximaler evaluations-Anzahl
        self.mu = mu #Eltern
        self.lamb = lamb #Kinder
        self.mutation_k = mutation_k #Für Mutationen k=2 oder k=4
        self.mutation_method = mutation_method #Mutationen
        self.mutationen = Mutationen()
        self.rekombination_method = recombination_method #Rekombinationen
        self.rekombination = Rekombination()
        self.zielfunktion = zielfunktion #Zielfunktion
        self.max_evaluations = max_evaluations  # Max Evaluations festgelegt
        self.population = [random.sample(range(1, 10), 9) for _ in range(mu)] #Erstellung von (mu)-Individuen (zufällige Zahl von 1 bis 9) einer zufälliger Population 


    def run(self):
        auswertungen = 0 #Anzahl der Auswertungen der Zielfunktion
        while auswertungen < self.max_evaluations:
            offspring = []
            for _ in range(self.lamb): #Schleife für Kinder
                parent1, parent2 = random.sample(self.population, 2)
                child = self.rekombination.recombine(parent1, parent2, self.rekombination_method) #Rekombination
                child = self.mutationen.mutate(child, self.mutation_method, self.mutation_k) #Mutation
                offspring.append(child)
                auswertungen += 1
            self.population += offspring
            self.population.sort(key=self.zielfunktion) #Sortierung nach Zielfunktion
            self.population = self.population[:self.mu]   #Auswahl bester Individuen

            if self.is_magic_square(self.population[0]):
                return auswertungen

    def is_magic_square(self, kandidat): #Überprüfen ob magischer Würfel passt
        return self.zielfunktion(kandidat) == 0 #Zielfunktion ist gleich 0 für magischen Würfel

if __name__ == "__main__":
    mu_values = [5]
    lambda_values = [1]
    mutation_operators = ['1-Translocation']
    mutation_k_werte = [4]
    rekombination_operators = ['OBX', 'PMX', 'NIX']
    zielfunktionen = Zielfunktionen()
    zielfunktion_operators = [zielfunktionen.ziel_1, zielfunktionen.ziel_2, zielfunktionen.ziel_3]

    max_evaluations = 10000

    results = []

    for mu in mu_values:
        for lamb in lambda_values:
            for mutation_operator in mutation_operators:
                for mutation_k in mutation_k_werte:
                    for rekombination_operator in rekombination_operators:
                        for zielfunktion_operator in zielfunktion_operators:
                            evals_for_combination = []
                            for _ in range(5):  # 100 Durchläufe für jede Kombination
                                random.seed()
                                ea = EvolutionaryAlgorithm(mu, lamb, mutation_k, mutation_operator, rekombination_operator, zielfunktion_operator, max_evaluations)
                                evaluations = ea.run()
                                if evaluations is not None:
                                    evals_for_combination.append(evaluations)
                            if evals_for_combination:
                                median_evaluations = statistics.median(evals_for_combination)
                                results.append((median_evaluations, mu, lamb, mutation_operator, mutation_k, rekombination_operator, zielfunktion_operator.__name__))
                                print(f"Median Evaluations: {median_evaluations}, Mu: {mu}, Lambda: {lamb}, Mutation: {mutation_operator}, Mutation K: {mutation_k}, Rekombination: {rekombination_operator}, Zielfunktion: {zielfunktion_operator.__name__}")

    # Sortiere die Ergebnisse nach der Anzahl der Evaluierungen (Median) und gib die besten und schlechtesten Parameterkombinationen aus
    results.sort()

    print("\nDie 5 besten Parameterkombinationen:")
    for result in results[:5]:
        print(result)

    print("\nDie 5 schlechtesten Parameterkombinationen:")
    for result in results[-5:]:
        print(result)

Median Evaluations: 4768.0, Mu: 5, Lambda: 1, Mutation: 1-Translocation, Mutation K: 4, Rekombination: OBX, Zielfunktion: ziel_1
Median Evaluations: 6169, Mu: 5, Lambda: 1, Mutation: 1-Translocation, Mutation K: 4, Rekombination: OBX, Zielfunktion: ziel_2
Median Evaluations: 6667, Mu: 5, Lambda: 1, Mutation: 1-Translocation, Mutation K: 4, Rekombination: OBX, Zielfunktion: ziel_3


KeyboardInterrupt: 